<img style="float: left; margin: 30px 15px 15px 15px;" src="https://pngimage.net/wp-content/uploads/2018/06/logo-iteso-png-5.png" width="300" height="500" /> 
    
    
### <font color='navy'> Simulación de procesos financieros. 

**Nombres:** <span style="color:firebrick"> Andrés Lares Barragán</span> | <span style="color:green"> Ivette Landaverde Mercado </span>

**Fecha:** 07 de mayo del 2021.  

**Expediente** : <span style="color:firebrick"> 724055 </span> | <span style="color:green"> 722626 </span>

**Profesor:** Oscar David Jaramillo Zuluaga.

# Tarea 11: Clase 24
    
### [Link de repositorio GitHub](https://github.com/AndresLaresBarragan/ProyectoConjunto_ALares_ILandaverde)

Realizar la valuación de las opciones barrera para las acciones de Microsoft, siguiendo los siguientes lineamientos:
1. Realizar la valuación para el tipo de opción barrera 'CALL'- down-and-out, usando la suposición de rendimientos normales y usando el método del histograma. Para ambas valuaciones, comparar los resultados de la evaluación obtenida con los resultados obtenidos por la valuación Europea. Como se realizó en clase.
2. Repetir el inciso anterior pero en esta ocasión para el caso de 'PUT'- Up-and-out.

**Nota:** Analice los datos descargados y según el método ya sea `down-and-out` o `Up-and-out` defina un strike price usando la volatility smile vista en la clase de opciones europeas y basado en el spot price y el strike defina una barrera adecuada.

Se habilitará un enlace en canvas para subir un cuaderno de python, con la solución.

**Criterio de revisión.**

> Se te calificará de acuerdo a los resultados finales que reportes, basados en tu análisis.

In [1]:
#pip install plotly

In [2]:
# importar paqueterías
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import datetime
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns
import plotly.graph_objects as go

### <span style="color:firebrick"> Andrés Lares Barragán</span>  


In [ ]:
# Solucion

### <span style="color:green"> Ivette Landaverde </span>

In [20]:
# Solución 